The dataset contains six features (first six cloumns) and a target variable (the 7th column) 
It can be downloaded from the drop box location.

In [2]:
# First step is to load the dataset to RDD.
from pyspark.sql.types import *
import urllib2

# read the content of the url
datafile = urllib2.urlopen('https://www.dropbox.com/s/v4g5fb5ctlf9p4a/solved_heuristic_2.txt?raw=1')
# get the lines into a vector
lines = [line.rstrip('\n') for line in datafile]

rdd = sc.parallelize(lines)
print rdd.first()

In [3]:
#  separate the label from the features
rdd1 = rdd.map(lambda l: l.split(","))

In [4]:
# convert the RDD to data frame
schema = StructType([
    StructField('f1', StringType(), True),
    StructField('f2', StringType(), True),
    StructField('f3', StringType(), True),
    StructField('f4', StringType(), True),
    StructField('f5', StringType(), True),
    StructField('f6', StringType(), True),
    StructField('label', StringType(), True)
  ])
df = sqlContext.createDataFrame(rdd1, schema)

In [5]:
#convert the character columns to numeric
df = df.withColumn("f1", df["f1"].cast(IntegerType()))
df = df.withColumn("f2", df["f2"].cast(IntegerType()))
df = df.withColumn("f3", df["f3"].cast(IntegerType()))
df = df.withColumn("f4", df["f4"].cast(IntegerType()))
df = df.withColumn("f5", df["f5"].cast(IntegerType()))
df = df.withColumn("f6", df["f6"].cast(IntegerType()))
df = df.withColumn("label", df["label"].cast(DoubleType()))

In [6]:
df
print df.dtypes
df.describe().show()
df.first()

In [7]:
#use a vector assembler to create features
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint

ignore = ['label']
VA = VectorAssembler(
    inputCols=[x for x in df.columns if x not in ignore],
    outputCol='features')

VA1 = VA.transform(df)
print VA1.first()

df1 = (VA1.select("label","features"))
print df1.first()

df2 = df1.map(lambda (l,f): LabeledPoint(l,f))
print df2.first()

In [8]:
# divide the data to training and testing
splits = df2.randomSplit([0.8, 0.2], 2569)
train_data = splits[0]
test_data = splits[1]


In [9]:
from pyspark.mllib.regression import LinearRegressionWithSGD

# Build the model
print type(train_data)

model = LinearRegressionWithSGD.train(train_data, iterations=10000, step=0.001)


In [10]:
# Evaluate the model on training data
valuesAndPreds = train_data.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

In [11]:
# Save and load model
model.save(sc, "tmpModel")
sameModel = LinearRegressionModel.load(sc, "tmpModel")

In [12]:
# use the model on the test data
Preds = test_data.map(lambda p: (p.label, model.predict(p.features)))
print Preds.first()
MSE_test = Preds.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / Preds.count()
print("Mean Squared Error = " + str(MSE_test))